In [1]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

import pandas as pd
import numpy as np

In [97]:
dataset = pd.read_excel('../data/4/result/dataset.xlsx')
dataset = dataset.set_index(['시군구', '시군구코드'])

dataset['도서관당_면적'] = dataset['면적'] / dataset['도서관수']
dataset['도서관당_도서'] = dataset['도서'] / dataset['도서관수']

dataset['일인당_대출권수'] = dataset['대출권수'] / dataset['총인구수']
# 시점을 더미 데이터로 변환 
dataset = pd.get_dummies(dataset, columns = ['시점'])
dataset = dataset.rename(columns={'15분이내_접근가능인구_비율_초등학교':'접근가능인구_비율_초등학교', '15분이내_접근가능인구_비율_공공의료시설':'접근가능인구_비율_공공의료시설'})

dataset.head(5)

,,시도,SDI,총인구수,영유아_어린이_인구_비율,청소년_인구_비율,고령인구_비율,여성인구비율,자연인구증가율,인위인구증가율,십만명당_문화기반시설수,...,대출권수_역사,대출권수,도서관당_면적,도서관당_도서,일인당_대출권수,시점_2014,시점_2015,시점_2016,시점_2017,시점_2018
시군구,시군구코드,,,,,,,,,,,,,,,,,,,,,
종로구,11010,서울특별시,-2.415031,153407.0,0.107036,0.059443,0.146506,0.504817,0.267263,-21.465774,37.6,...,144547.0,1158860.0,4.782594e+06,210773.000000,7.554153,1,0,0,0,0
중구,11020,서울특별시,-2.362601,125273.0,0.103538,0.047456,0.145291,0.500714,2.857759,-22.886017,18.0,...,50810.0,480436.0,2.490046e+06,92864.250000,3.835112,1,0,0,0,0
용산구,11030,서울특별시,-2.640132,229055.0,0.119022,0.051581,0.137262,0.513246,3.737094,-21.903036,5.9,...,61587.0,615193.0,7.289596e+06,258421.666667,2.685787,1,0,0,0,0
성동구,11040,서울특별시,-1.988008,292786.5,0.122975,0.053449,0.118708,0.502542,4.744071,-16.489831,3.7,...,353931.0,3586468.0,2.809498e+06,74531.000000,12.249431,1,0,0,0,0
광진구,11050,서울특별시,-1.823281,361146.5,0.118682,0.060341,0.102618,0.508316,4.466332,-20.520758,2.8,...,114378.0,1048389.0,5.687544e+06,86164.000000,2.902947,1,0,0,0,0


In [98]:
dataset.dtypes

시도                        object
SDI                      float64
총인구수                     float64
영유아_어린이_인구_비율            float64
청소년_인구_비율                float64
고령인구_비율                  float64
여성인구비율                   float64
자연인구증가율                  float64
인위인구증가율                  float64
십만명당_문화기반시설수             float64
15분이내_접근가능인구_비율_고등학교     float64
접근가능인구_비율_공공의료시설         float64
15분이내_접근가능인구_비율_대규모점포    float64
15분이내_접근가능인구_비율_병·의원     float64
15분이내_접근가능인구_비율_전통시장     float64
15분이내_접근가능인구_비율_종합병원     float64
15분이내_접근가능인구_비율_중학교      float64
접근가능인구_비율_초등학교           float64
재정자립도                    float64
면적                       float64
공원면적                     float64
도서관수                       int64
시설_연면적                   float64
개관일수                     float64
AGE                      float64
직원_정규직                     int64
도서관총예산                     int64
도서_총류                      int64
도서_철학                      int64
도서_종교                      int64
도서_사회과학   

In [99]:
dataset.columns

Index(['시도', 'SDI', '총인구수', '영유아_어린이_인구_비율', '청소년_인구_비율', '고령인구_비율', '여성인구비율',
       '자연인구증가율', '인위인구증가율', '십만명당_문화기반시설수', '15분이내_접근가능인구_비율_고등학교',
       '접근가능인구_비율_공공의료시설', '15분이내_접근가능인구_비율_대규모점포', '15분이내_접근가능인구_비율_병·의원',
       '15분이내_접근가능인구_비율_전통시장', '15분이내_접근가능인구_비율_종합병원', '15분이내_접근가능인구_비율_중학교',
       '접근가능인구_비율_초등학교', '재정자립도', '면적', '공원면적', '도서관수', '시설_연면적', '개관일수',
       'AGE', '직원_정규직', '도서관총예산', '도서_총류', '도서_철학', '도서_종교', '도서_사회과학',
       '도서_순수과학', '도서_기술과학', '도서_예술', '도서_언어', '도서_문학', '도서_역사', '도서_국내',
       '도서_국외', '도서', '대출권수_총류', '대출권수_철학', '대출권수_종교', '대출권수_사회과학',
       '대출권수_순수과학', '대출권수_기술과학', '대출권수_예술', '대출권수_언어', '대출권수_문학', '대출권수_역사',
       '대출권수', '도서관당_면적', '도서관당_도서', '일인당_대출권수', '시점_2014', '시점_2015',
       '시점_2016', '시점_2017', '시점_2018'],
      dtype='object')

In [251]:
regression_dataset = dataset[['시도', 'SDI', '총인구수', '영유아_어린이_인구_비율', '고령인구_비율', '여성인구비율', '자연인구증가율', '인위인구증가율', '십만명당_문화기반시설수', '접근가능인구_비율_공공의료시설', '재정자립도', '도서관당_면적', '도서','AGE', '시점_2014', '시점_2015', '시점_2016', '시점_2017', '시점_2018','공원면적', '재정자립도','직원_정규직', '일인당_대출권수']]
regression_dataset.head()

,,시도,SDI,총인구수,영유아_어린이_인구_비율,고령인구_비율,여성인구비율,자연인구증가율,인위인구증가율,십만명당_문화기반시설수,접근가능인구_비율_공공의료시설,...,AGE,시점_2014,시점_2015,시점_2016,시점_2017,시점_2018,공원면적,재정자립도,직원_정규직,일인당_대출권수
시군구,시군구코드,,,,,,,,,,,,,,,,,,,,,
종로구,11010,서울특별시,-2.415031,153407.0,0.107036,0.146506,0.504817,0.267263,-21.465774,37.6,36.311895,...,34.200000,1,0,0,0,0,305048.7,55.0,120,7.554153
중구,11020,서울특별시,-2.362601,125273.0,0.103538,0.145291,0.500714,2.857759,-22.886017,18.0,73.204035,...,4.500000,1,0,0,0,0,438409.8,63.5,49,3.835112
용산구,11030,서울특별시,-2.640132,229055.0,0.119022,0.137262,0.513246,3.737094,-21.903036,5.9,16.461084,...,44.333333,1,0,0,0,0,101827.9,43.1,84,2.685787
성동구,11040,서울특별시,-1.988008,292786.5,0.122975,0.118708,0.502542,4.744071,-16.489831,3.7,41.522957,...,7.333333,1,0,0,0,0,488313.1,32.2,31,12.249431
광진구,11050,서울특별시,-1.823281,361146.5,0.118682,0.102618,0.508316,4.466332,-20.520758,2.8,39.066920,...,9.666667,1,0,0,0,0,720460.8,30.4,37,2.902947


In [252]:
scale_var = []
categoty_var = ['시점_2014', '시점_2015','시점_2016', '시점_2017', '시점_2018']
just_var = ['SDI', '도서', '총인구수', '영유아_어린이_인구_비율', '고령인구_비율', '여성인구비율', '인위인구증가율', '직원_정규직', '접근가능인구_비율_공공의료시설']

In [258]:
cate_fomul = ' + '.join(["({})".format(name) for name in categoty_var])
cate_fomul

'(시점_2014) + (시점_2015) + (시점_2016) + (시점_2017) + (시점_2018)'

In [259]:
scale_fomul = ' + '.join(["scale({})".format(name) for name in scale_var])
scale_fomul

''

In [260]:
just_fomul = ' + '.join(["({})".format(name) for name in just_var])
just_fomul

'(SDI) + (도서) + (총인구수) + (영유아_어린이_인구_비율) + (고령인구_비율) + (여성인구비율) + (인위인구증가율) + (직원_정규직) + (접근가능인구_비율_공공의료시설)'

In [261]:
indepent_fomul =  just_fomul+ ' + '  + cate_fomul 
indepent_fomul

'(SDI) + (도서) + (총인구수) + (영유아_어린이_인구_비율) + (고령인구_비율) + (여성인구비율) + (인위인구증가율) + (직원_정규직) + (접근가능인구_비율_공공의료시설) + (시점_2014) + (시점_2015) + (시점_2016) + (시점_2017) + (시점_2018)'

In [262]:
# mixed effect model
md = smf.mixedlm("일인당_대출권수 ~ "+indepent_fomul, regression_dataset, groups=regression_dataset["시도"])

mdf = md.fit()

print(mdf.summary())

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with cg
  warnings.warn(


            Mixed Linear Model Regression Results
Model:               MixedLM  Dependent Variable:  일인당_대출권수  
No. Observations:    1145     Method:              REML      
No. Groups:          17       Scale:               0.6608    
Min. group size:     5        Log-Likelihood:      -1434.2913
Max. group size:     155      Converged:           No        
Mean group size:     67.4                                    
-------------------------------------------------------------
                  Coef.  Std.Err.    z    P>|z| [0.025 0.975]
-------------------------------------------------------------
Intercept        -72.670                                     
SDI               -0.259    0.040  -6.509 0.000 -0.337 -0.181
도서                 0.000    0.000   9.720 0.000  0.000  0.000
총인구수              -0.000    0.000 -21.668 0.000 -0.000 -0.000
영유아_어린이_인구_비율      4.232    2.108   2.007 0.045  0.100  8.364
고령인구_비율           -1.093    1.173  -0.932 0.351 -3.392  1.205
여성인구비율            15

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2206: ConvergenceWarning: MixedLM optimization failed, trying a different optimizer may help.
  warnings.warn(msg, ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2218: ConvergenceWarning: Gradient optimization failed, |grad| = 67.379157
  warnings.warn(msg, ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2711: RuntimeWarning: invalid value encountered in sqrt
  sdf[0:self.k_fe, 1] = np.sqrt(np.diag(self.cov_params()[0:self.k_fe]))
